In [2]:
from google.colab import drive
drive.mount("/content/drive")

Mounted at /content/drive


In [3]:
!python -m pip install evaluate

INFO: pip is looking at multiple versions of multiprocess to determine which version is compatible with other requirements. This could take a while.
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 84.0/84.0 kB 4.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 471.6/471.6 kB 18.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 116.3/116.3 kB 11.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 134.8/134.8 kB 12.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 194.1/194.1 kB 16.8 MB/s eta 0:00:00


In [4]:
!python -m pip install bert_score

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 61.1/61.1 kB 2.9 MB/s eta 0:00:00


In [5]:
from evaluate import load

In [6]:
!python -m pip install sacrebleu

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 51.8/51.8 kB 3.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 104.0/104.0 kB 7.1 MB/s eta 0:00:00


In [7]:
!python -m pip install rouge

In [8]:
# metrics:
bleu = load("bleu")
sbleu = load("sacrebleu")
meteor = load("meteor")

/usr/local/lib/python3.10/dist-packages/huggingface_hub/utils/_token.py:89: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


[nltk_data] Downloading package wordnet to /root/nltk_data...
[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Unzipping tokenizers/punkt.zip.
[nltk_data] Downloading package omw-1.4 to /root/nltk_data...


In [9]:
bertscore = load("bertscore")

In [10]:
import os
os.chdir("/content/drive/MyDrive/RuRACE/OrganizedOutputData")

In [11]:
import pandas as pd
from typing import List, Any
from rouge import Rouge

In [12]:
rouge = Rouge()

In [13]:
def compute_metrics(output: List[str], label_batch: List[str]) -> dict:
    rouge_scores = []
    for sent_a, sent_b in zip(output, label_batch):
      try:
        scores_ = rouge.get_scores(sent_a, sent_b)
        scores = {
            "rouge1": scores_[0]["rouge-1"]["f"],
            "rouge2": scores_[0]["rouge-2"]["f"],
            "rougeL": scores_[0]["rouge-l"]["f"]
        }
      except Exception as exc:
        print(exc)
        scores = {
            "rouge1": 0.0, "rouge2": 0.0, "rougeL": 0.0
        }
      rouge_scores.append(scores)

    rouge_scores = pd.DataFrame(rouge_scores).mean()

    metric_dict = {
        "bleu1": bleu.compute(predictions=output, references=[[label] for label in label_batch], max_order=1)["bleu"],
        "bleu2": bleu.compute(predictions=output, references=[[label] for label in label_batch], max_order=2)["bleu"],
        "bleu3": bleu.compute(predictions=output, references=[[label] for label in label_batch], max_order=3)["bleu"],
        "bleu4": bleu.compute(predictions=output, references=[[label] for label in label_batch], max_order=4)["bleu"],
        "sbleu": sbleu.compute(predictions=output, references=[[label] for label in label_batch])["score"],
        "rouge1": rouge_scores["rouge1"],
        "rouge2": rouge_scores["rouge2"],
        "rougeL": rouge_scores["rougeL"],
        "meteor": meteor.compute(predictions=output, references=label_batch)["meteor"],
        "bertscore": pd.DataFrame(bertscore.compute(
                        predictions=output,
                        references=label_batch,
                        lang="ru", verbose=True,
                        model_type = "bert-base-multilingual-cased"
                    ))["f1"].mean()
    }
    return metric_dict

In [14]:
def recalculate_metrics(input_addr: str) -> Any:
    if input_addr.endswith(".xlsx"):
      df = pd.read_excel(input_addr, engine="openpyxl")
    elif input_addr.endswith(".csv"):
      df = pd.read_csv(input_addr, sep=";")

    scores = compute_metrics(
       output = df["output"].values,
       label_batch = df["distractors"].values
    )

    return scores

In [15]:
recalculate_metrics("Ru-RACE-TITLE/RuGPT3Metrics-Title.xlsx")

tokenizer_config.json:   0%|          | 0.00/49.0 [00:00<?, ?B/s]

config.json:   0%|          | 0.00/625 [00:00<?, ?B/s]

vocab.txt:   0%|          | 0.00/996k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/1.96M [00:00<?, ?B/s]

/usr/local/lib/python3.10/dist-packages/transformers/tokenization_utils_base.py:1601: FutureWarning: `clean_up_tokenization_spaces` was not set. It will be set to `True` by default. This behavior will be depracted in transformers v4.45, and will be then set to `False` by default. For more details check this issue: https://github.com/huggingface/transformers/issues/31884
  warnings.warn(


...

  0%|          | 0/8 [00:00<?, ?it/s]

computing greedy matching.


  0%|          | 0/4 [00:00<?, ?it/s]

done in 62.83 seconds, 3.85 sentences/sec


{'bleu1': 0.14573643410852713,
 'bleu2': 0.08214952774730294,
 'bleu3': 0.05059941724165758,
 'bleu4': 0.031855833737756876,
 'sbleu': 3.185583373775689,
 'rouge1': 0.12793514645224374,
 'rouge2': 0.04777516368199007,
 'rougeL': 0.12604396640903068,
 'meteor': 0.12414470652609082,
 'bertscore': 0.6868281652612135}

In [16]:
recalculate_metrics("Ru-RACE-TITLE/RuGPT3Metrics-Title-val.xlsx")

calculating scores...
computing bert embedding.


  0%|          | 0/7 [00:00<?, ?it/s]

computing greedy matching.


  0%|          | 0/4 [00:00<?, ?it/s]

done in 88.85 seconds, 2.46 sentences/sec


{'bleu1': 0.14273504273504273,
 'bleu2': 0.07764415424612764,
 'bleu3': 0.05184237488627642,
 'bleu4': 0.03828700476955019,
 'sbleu': 3.8287004769550177,
 'rouge1': 0.1272314147472741,
 'rouge2': 0.048690815182771525,
 'rougeL': 0.1232167896397365,
 'meteor': 0.12779555130517206,
 'bertscore': 0.6871789625790565}

In [17]:
recalculate_metrics("Ru-RACE-TITLE/T5Metrics-Title.xlsx")

calculating scores...
computing bert embedding.


  0%|          | 0/8 [00:00<?, ?it/s]

computing greedy matching.


  0%|          | 0/4 [00:00<?, ?it/s]

done in 69.20 seconds, 3.50 sentences/sec


{'bleu1': 0.4911451055375933,
 'bleu2': 0.38800164269377985,
 'bleu3': 0.30927765808563357,
 'bleu4': 0.2296233094832111,
 'sbleu': 22.9623309483211,
 'rouge1': 0.16943836502026202,
 'rouge2': 0.07605548455902593,
 'rougeL': 0.16211002076352524,
 'meteor': 0.4534923034232536,
 'bertscore': 0.7871606608560263}

In [18]:
recalculate_metrics("Ru-RACE-TITLE/T5Metrics-Title-val.xlsx")

calculating scores...
computing bert embedding.


  0%|          | 0/7 [00:00<?, ?it/s]

computing greedy matching.


  0%|          | 0/4 [00:00<?, ?it/s]

done in 94.69 seconds, 2.31 sentences/sec


{'bleu1': 0.5085678851379524,
 'bleu2': 0.40521273728190876,
 'bleu3': 0.32788226385095254,
 'bleu4': 0.25173273892910475,
 'sbleu': 25.173273892910473,
 'rouge1': 0.17266652039482724,
 'rouge2': 0.08759229773762006,
 'rougeL': 0.1679471474582616,
 'meteor': 0.46094658835033037,
 'bertscore': 0.7908510348023889}

In [19]:
recalculate_metrics("Ru-RACE-TITLE/RuGPT3Metrics-Title-Baseline.xlsx")




















calculating scores...
computing bert embedding.


  0%|          | 0/7 [00:00<?, ?it/s]

computing greedy matching.


  0%|          | 0/4 [00:00<?, ?it/s]

done in 74.25 seconds, 3.26 sentences/sec


{'bleu1': 0.0766458433445459,
 'bleu2': 0.03219135110609606,
 'bleu3': 0.01409588112576423,
 'bleu4': 0.005322846339104705,
 'sbleu': 0.5322846339104708,
 'rouge1': 0.04548615368679221,
 'rouge2': 0.012528522415778565,
 'rougeL': 0.044747360799238994,
 'meteor': 0.05567730510521785,
 'bertscore': 0.6246121609506529}

In [20]:
recalculate_metrics("Ru-RACE-TITLE/RuGPT3Metrics-Title-Baseline-val.xlsx")

















calculating scores...
computing bert embedding.


  0%|          | 0/7 [00:00<?, ?it/s]

computing greedy matching.


  0%|          | 0/4 [00:00<?, ?it/s]

done in 99.43 seconds, 2.20 sentences/sec


{'bleu1': 0.07136929460580914,
 'bleu2': 0.02633472765086326,
 'bleu3': 0.010920587849528747,
 'bleu4': 0.004562336023002951,
 'sbleu': 0.4562336023002954,
 'rouge1': 0.0451722044220349,
 'rouge2': 0.009067824306886345,
 'rougeL': 0.043132676043347334,
 'meteor': 0.0536739954371458,
 'bertscore': 0.6271514484327133}

In [21]:
recalculate_metrics("Ru-RACE-TF/RuGPT3Metrics-TF.xlsx")

calculating scores...
computing bert embedding.


  0%|          | 0/6 [00:00<?, ?it/s]

computing greedy matching.


  0%|          | 0/3 [00:00<?, ?it/s]

done in 134.69 seconds, 1.39 sentences/sec


{'bleu1': 0.2392745029647715,
 'bleu2': 0.11203807103172526,
 'bleu3': 0.06976615997435474,
 'bleu4': 0.048885246357355264,
 'sbleu': 4.888524635735528,
 'rouge1': 0.14183032148772903,
 'rouge2': 0.04298818755327981,
 'rougeL': 0.13795784771775013,
 'meteor': 0.1684406242277731,
 'bertscore': 0.7022977289669017}

In [22]:
recalculate_metrics("Ru-RACE-TF/RuGPT3Metrics-TF-val.xlsx")

calculating scores...
computing bert embedding.


  0%|          | 0/6 [00:00<?, ?it/s]

computing greedy matching.


  0%|          | 0/3 [00:00<?, ?it/s]

done in 149.98 seconds, 1.17 sentences/sec


{'bleu1': 0.2623415925855873,
 'bleu2': 0.14663753931679968,
 'bleu3': 0.1065836688919394,
 'bleu4': 0.08754344773730958,
 'sbleu': 8.754344773730962,
 'rouge1': 0.1662240227911487,
 'rouge2': 0.07044411153732268,
 'rougeL': 0.161641919161681,
 'meteor': 0.1891954770105611,
 'bertscore': 0.7101252147129604}

In [23]:
recalculate_metrics("Ru-RACE-TF/T5Metrics-TF.xlsx")

calculating scores...
computing bert embedding.


  0%|          | 0/6 [00:00<?, ?it/s]

computing greedy matching.


  0%|          | 0/3 [00:00<?, ?it/s]

done in 140.99 seconds, 1.33 sentences/sec


{'bleu1': 0.4379863216277702,
 'bleu2': 0.34442317767462716,
 'bleu3': 0.27722032910121286,
 'bleu4': 0.22426682631819161,
 'sbleu': 22.426682631819162,
 'rouge1': 0.25695910672850264,
 'rouge2': 0.089966486543421,
 'rougeL': 0.2530266769742387,
 'meteor': 0.427460095255116,
 'bertscore': 0.762440465669581}

In [24]:
recalculate_metrics("Ru-RACE-TF/T5Metrics-TF-val.xlsx")

calculating scores...
computing bert embedding.


  0%|          | 0/6 [00:00<?, ?it/s]

computing greedy matching.


  0%|          | 0/3 [00:00<?, ?it/s]

done in 155.93 seconds, 1.12 sentences/sec


{'bleu1': 0.4648949726989707,
 'bleu2': 0.376304329718654,
 'bleu3': 0.3123479273940566,
 'bleu4': 0.26361540613060575,
 'sbleu': 26.361540613060573,
 'rouge1': 0.2855607932447794,
 'rouge2': 0.12901542897599458,
 'rougeL': 0.2836442766014268,
 'meteor': 0.4484071726105385,
 'bertscore': 0.7707066117014204}

In [25]:
recalculate_metrics("Ru-RACE-TF/RuGPT3Metrics-TF-Baseline.xlsx")


calculating scores...
computing bert embedding.


  0%|          | 0/6 [00:00<?, ?it/s]

computing greedy matching.


  0%|          | 0/3 [00:00<?, ?it/s]

done in 147.43 seconds, 1.27 sentences/sec


{'bleu1': 0.14941939890710382,
 'bleu2': 0.05576879588346384,
 'bleu3': 0.028310101530625364,
 'bleu4': 0.017318225912933596,
 'sbleu': 1.7318225912933605,
 'rouge1': 0.08529671018983287,
 'rouge2': 0.017611553186290835,
 'rougeL': 0.08291039955509942,
 'meteor': 0.09435241991808294,
 'bertscore': 0.6403511996893959}

In [26]:
recalculate_metrics("Ru-RACE-TF/RuGPT3Metrics-TF-Baseline-val.xlsx")


calculating scores...
computing bert embedding.


  0%|          | 0/6 [00:00<?, ?it/s]

computing greedy matching.


  0%|          | 0/3 [00:00<?, ?it/s]

done in 163.10 seconds, 1.07 sentences/sec


{'bleu1': 0.14981915096135542,
 'bleu2': 0.05404492454137925,
 'bleu3': 0.024925520648322097,
 'bleu4': 0.012303888982073522,
 'sbleu': 1.2303888982073525,
 'rouge1': 0.0854662594254064,
 'rouge2': 0.016669269419184313,
 'rougeL': 0.08302963458819616,
 'meteor': 0.09535941909771224,
 'bertscore': 0.6363820409774781}

In [27]:
recalculate_metrics("MuSeRC/metrics_muserc_val_gpt3.csv")





object of type 'float' has no len()























calculating scores...
computing bert embedding.


  0%|          | 0/16 [00:00<?, ?it/s]

computing greedy matching.


  0%|          | 0/9 [00:00<?, ?it/s]

done in -186.97 seconds, -2.83 sentences/sec


{'bleu1': 0.3923254579172493,
 'bleu2': 0.2965467349805479,
 'bleu3': 0.20650828994482218,
 'bleu4': 0.12478810773476903,
 'sbleu': 12.478810773476901,
 'rouge1': 0.21797239170071422,
 'rouge2': 0.038737662311673936,
 'rougeL': 0.2176848279726968,
 'meteor': 0.40875158792380617,
 'bertscore': 0.7603664896213265}

In [28]:
recalculate_metrics("MuSeRC/metrics_muserc_val_t5.csv")

calculating scores...
computing bert embedding.


  0%|          | 0/17 [00:00<?, ?it/s]

computing greedy matching.


  0%|          | 0/9 [00:00<?, ?it/s]

done in -181.34 seconds, -2.91 sentences/sec


{'bleu1': 0.4675939627108612,
 'bleu2': 0.3761125861338263,
 'bleu3': 0.2967187837594886,
 'bleu4': 0.2361960922355481,
 'sbleu': 23.61960922355481,
 'rouge1': 0.26027819813396824,
 'rouge2': 0.04452132820741295,
 'rougeL': 0.25972461661447765,
 'meteor': 0.45778884956231264,
 'bertscore': 0.7602250056736397}

In [29]:
recalculate_metrics("MuSeRC/metrics_muserc_val_gpt3_baseline.csv")

calculating scores...
computing bert embedding.


  0%|          | 0/17 [00:00<?, ?it/s]

computing greedy matching.


  0%|          | 0/9 [00:00<?, ?it/s]

done in -175.59 seconds, -3.01 sentences/sec


{'bleu1': 0.18387983506774003,
 'bleu2': 0.09668613466893236,
 'bleu3': 0.06686725423572143,
 'bleu4': 0.051600650760614895,
 'sbleu': 5.160065076061487,
 'rouge1': 0.0701266129116946,
 'rouge2': 0.023106334432506048,
 'rougeL': 0.06805398228245753,
 'meteor': 0.11254588562040088,
 'bertscore': 0.629059456175927}

In [31]:
recalculate_metrics("EGE/MetricsEGERuGPT3.xlsx")

calculating scores...
computing bert embedding.


  0%|          | 0/2 [00:00<?, ?it/s]

computing greedy matching.


  0%|          | 0/1 [00:00<?, ?it/s]

done in 399.01 seconds, 0.14 sentences/sec


{'bleu1': 0.15109646758962655,
 'bleu2': 0.03573413439120211,
 'bleu3': 0.008229663957034086,
 'bleu4': 0.0,
 'sbleu': 0.334901167417257,
 'rouge1': 0.07077599682391225,
 'rouge2': 0.003093495751243604,
 'rougeL': 0.06834238535994547,
 'meteor': 0.09216526521505411,
 'bertscore': 0.6584029739553278}

In [32]:
recalculate_metrics("EGE/MetricsEGET5.xlsx")

calculating scores...
computing bert embedding.


  0%|          | 0/2 [00:00<?, ?it/s]

computing greedy matching.


  0%|          | 0/1 [00:00<?, ?it/s]

done in 400.87 seconds, 0.14 sentences/sec


{'bleu1': 0.2910804309453767,
 'bleu2': 0.20925626319319682,
 'bleu3': 0.15663900362299318,
 'bleu4': 0.11636893477091932,
 'sbleu': 11.636893477091931,
 'rouge1': 0.13911496543657298,
 'rouge2': 0.005787687715792609,
 'rougeL': 0.13546726833793715,
 'meteor': 0.29606985142683745,
 'bertscore': 0.7106287338516929}

In [33]:
recalculate_metrics("EGE/metrics_muserc_ege_gpt3.csv")

calculating scores...
computing bert embedding.


  0%|          | 0/2 [00:00<?, ?it/s]

computing greedy matching.


  0%|          | 0/1 [00:00<?, ?it/s]

done in 402.12 seconds, 0.14 sentences/sec


{'bleu1': 0.09556611401615167,
 'bleu2': 0.01688615033612369,
 'bleu3': 0.004806071940936653,
 'bleu4': 0.0,
 'sbleu': 0.21803798747104558,
 'rouge1': 0.04400559059717733,
 'rouge2': 0.00261681467248827,
 'rougeL': 0.04347083123888856,
 'meteor': 0.06650551286782584,
 'bertscore': 0.6161898569627242}

In [34]:
recalculate_metrics("EGE/metrics_muserc_ege_t5.csv")

calculating scores...
computing bert embedding.


  0%|          | 0/2 [00:00<?, ?it/s]

computing greedy matching.


  0%|          | 0/1 [00:00<?, ?it/s]

done in 403.37 seconds, 0.14 sentences/sec


{'bleu1': 0.10295886540207737,
 'bleu2': 0.021992846391195968,
 'bleu3': 0.005535168930127735,
 'bleu4': 0.0,
 'sbleu': 0.23620087067069037,
 'rouge1': 0.04863871191636276,
 'rouge2': 0.002164502011173612,
 'rougeL': 0.04659958528707219,
 'meteor': 0.07770784143583405,
 'bertscore': 0.6263418479399248}

In [35]:
recalculate_metrics("EGE/metrics_baseline_ege_gpt3.csv")

calculating scores...
computing bert embedding.


  0%|          | 0/2 [00:00<?, ?it/s]

computing greedy matching.


  0%|          | 0/1 [00:00<?, ?it/s]

done in 405.05 seconds, 0.14 sentences/sec


{'bleu1': 0.11220196353436185,
 'bleu2': 0.0200287569032708,
 'bleu3': 0.005269226250789858,
 'bleu4': 0.0,
 'sbleu': 0.2284208375117482,
 'rouge1': 0.049963064002776066,
 'rouge2': 0.001471562440319376,
 'rougeL': 0.04740375442464361,
 'meteor': 0.07568684927388267,
 'bertscore': 0.5571857891299508}

In [36]:
recalculate_metrics("BartDG-EGE/BartDGOutputEGE.xlsx")

calculating scores...
computing bert embedding.


  0%|          | 0/2 [00:00<?, ?it/s]

computing greedy matching.


  0%|          | 0/1 [00:00<?, ?it/s]

done in 407.12 seconds, 0.14 sentences/sec


{'bleu1': 0.2666272581836141,
 'bleu2': 0.19439065212712237,
 'bleu3': 0.14641594540354982,
 'bleu4': 0.10775226564025675,
 'sbleu': 10.775226564025676,
 'rouge1': 0.12664267379530098,
 'rouge2': 0.007268746511164038,
 'rougeL': 0.12574466936071116,
 'meteor': 0.2776595130987213,
 'bertscore': 0.7082681298255921}

In [37]:
recalculate_metrics("BartDG-EGE/BartDG_PM_OutputEGE.xlsx")

calculating scores...
computing bert embedding.


  0%|          | 0/2 [00:00<?, ?it/s]

computing greedy matching.


  0%|          | 0/1 [00:00<?, ?it/s]

done in 408.66 seconds, 0.13 sentences/sec


{'bleu1': 0.28522058256513055,
 'bleu2': 0.202466972819592,
 'bleu3': 0.15066064499303958,
 'bleu4': 0.11024783482444073,
 'sbleu': 11.024783482444072,
 'rouge1': 0.12733462126635603,
 'rouge2': 0.0031351574557751173,
 'rougeL': 0.12402701340372424,
 'meteor': 0.28472548140884046,
 'bertscore': 0.7089641375975175}

In [38]:
recalculate_metrics("BartDG-EGE/BartDG_ANPM_OutputEGE.xlsx")

calculating scores...
computing bert embedding.


  0%|          | 0/2 [00:00<?, ?it/s]

computing greedy matching.


  0%|          | 0/1 [00:00<?, ?it/s]

done in 410.25 seconds, 0.13 sentences/sec


{'bleu1': 0.2739325756628323,
 'bleu2': 0.19706366903228534,
 'bleu3': 0.14638702529794567,
 'bleu4': 0.1070538228616483,
 'sbleu': 10.705382286164834,
 'rouge1': 0.11850034335398428,
 'rouge2': 0.002140802496342796,
 'rougeL': 0.11747091026713492,
 'meteor': 0.27779385962026554,
 'bertscore': 0.7062055024233731}